<a href="https://colab.research.google.com/github/Vedmani/wear-particle-classification/blob/main/ResNet101.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install wandb -qU
!wandb login

     |████████████████████████████████| 1.9 MB 32.1 MB/s 
     |████████████████████████████████| 162 kB 66.6 MB/s 
     |████████████████████████████████| 182 kB 64.3 MB/s 
     |████████████████████████████████| 63 kB 1.6 MB/s 
     |████████████████████████████████| 162 kB 68.4 MB/s 
     |████████████████████████████████| 158 kB 69.3 MB/s 
     |████████████████████████████████| 157 kB 51.5 MB/s 
     |████████████████████████████████| 157 kB 71.3 MB/s 
     |████████████████████████████████| 157 kB 71.9 MB/s 
     |████████████████████████████████| 157 kB 71.7 MB/s 
     |████████████████████████████████| 157 kB 74.9 MB/s 
     |████████████████████████████████| 157 kB 67.7 MB/s 
     |████████████████████████████████| 157 kB 67.5 MB/s 
     |████████████████████████████████| 156 kB 73.2 MB/s 
wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wand

In [3]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
import wandb
from wandb.keras import WandbCallback

In [4]:
tf.keras.utils.set_random_seed(123)
tf.config.experimental.enable_op_determinism()

In [5]:
PATH = '/content/drive/MyDrive/Research Internship/wear particles'
batch_size = 32
img_height = 224
img_width = 224

train_ds = tf.keras.utils.image_dataset_from_directory(
  PATH,
  validation_split=0.2,
  subset="training",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

val_ds = tf.keras.utils.image_dataset_from_directory(
  PATH,
  validation_split=0.2,
  subset="validation",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

class_names = train_ds.class_names
print(class_names)
num_classes = len(class_names)

Found 740 files belonging to 5 classes.
Using 592 files for training.
Found 740 files belonging to 5 classes.
Using 148 files for validation.
['cutting wear', 'normal rubbign and cutting wear combined', 'normal rubbing wear', 'red oxide', 'severe sliding wear']


In [6]:
resnet101 = tf.keras.applications.ResNet101(
    include_top=False,
    weights="imagenet",
    input_tensor=None,
    input_shape=None,
    pooling=None)
data_augmentation = tf.keras.Sequential([layers.RandomFlip("horizontal_and_vertical"),layers.RandomRotation(0.2),])
model_resnet101=Sequential([
  #data_augmentation,
  layers.Rescaling(1./255, input_shape=(img_height, img_width, 3)),
  resnet101,
  layers.Flatten(),
  layers.Dense(1028, activation='relu'),
  #layers.Dropout(0.5), 
  layers.Dense(num_classes, activation='softmax')
])
resnet101.trainable=False
model_resnet101.summary()

171458560/171446536 [==============================] - 1s 0us/step
Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 rescaling (Rescaling)       (None, 224, 224, 3)       0         
                                                                 
 resnet101 (Functional)      (None, None, None, 2048)  42658176  
                                                                 
 flatten (Flatten)           (None, 100352)            0         
                                                                 
 dense (Dense)               (None, 1028)              103162884 
                                                                 
 dense_1 (Dense)             (None, 5)                 5145      
                                                                 
Total params: 145,826,205
Trainable params: 103,168,029
Non-trainable params: 42,658,176
______________________________

In [7]:
model_resnet101.compile(
              optimizer= tf.keras.optimizers.Adam(learning_rate=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-07, amsgrad=False, name='Adam'),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
              metrics=['accuracy']
              )

In [8]:
wandb.init(
      # Set the project where this run will be logged
      project="Transfer_learning", 
      name = "resnet101",
      notes = "without augment and without dropout"
)

ERROR:wandb.jupyter:Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: vedmanivaidya. Use `wandb login --relogin` to force relogin


In [9]:
%%time
epochs=40
history = model_resnet101.fit(
  train_ds,
  validation_data=val_ds,
  epochs=epochs,
callbacks=[WandbCallback()]
)

wandb: WARNING The save_model argument by default saves the model in the HDF5 format that cannot save custom objects like subclassed models and custom layers. This behavior will be deprecated in a future release in favor of the SavedModel format. Meanwhile, the HDF5 model is saved as W&B files and the SavedModel as W&B Artifacts.


Epoch 1/40
19/19 [==============================] - ETA: 0s - loss: 19.6445 - accuracy: 0.4088

wandb: Adding directory to artifact (/content/wandb/run-20221012_075756-1fya4i00/files/model-best)... Done. 8.6s


19/19 [==============================] - 111s 4s/step - loss: 19.6445 - accuracy: 0.4088 - val_loss: 3.9181 - val_accuracy: 0.5000
Epoch 2/40
19/19 [==============================] - ETA: 0s - loss: 2.5671 - accuracy: 0.4932

wandb: Adding directory to artifact (/content/wandb/run-20221012_075756-1fya4i00/files/model-best)... Done. 8.4s


...

19/19 [==============================] - 72s 4s/step - loss: 0.3878 - accuracy: 0.8598 - val_loss: 0.4794 - val_accuracy: 0.8176
Epoch 21/40
19/19 [==============================] - 8s 323ms/step - loss: 0.3373 - accuracy: 0.8834 - val_loss: 0.5541 - val_accuracy: 0.8041
Epoch 22/40
19/19 [==============================] - 8s 307ms/step - loss: 0.3445 - accuracy: 0.8818 - val_loss: 0.5761 - val_accuracy: 0.7973
Epoch 23/40
19/19 [==============================] - ETA: 0s - loss: 0.3592 - accuracy: 0.8868

wandb: Adding directory to artifact (/content/wandb/run-20221012_075756-1fya4i00/files/model-best)... Done. 8.4s


19/19 [==============================] - 67s 4s/step - loss: 0.3592 - accuracy: 0.8868 - val_loss: 0.4612 - val_accuracy: 0.8378
Epoch 24/40
19/19 [==============================] - 8s 336ms/step - loss: 0.3969 - accuracy: 0.8564 - val_loss: 0.6499 - val_accuracy: 0.8243
Epoch 25/40
19/19 [==============================] - 7s 296ms/step - loss: 0.3083 - accuracy: 0.8953 - val_loss: 0.5345 - val_accuracy: 0.7838
Epoch 26/40
19/19 [==============================] - 7s 296ms/step - loss: 0.2779 - accuracy: 0.9155 - val_loss: 0.5792 - val_accuracy: 0.7635
Epoch 27/40
19/19 [==============================] - 7s 299ms/step - loss: 0.3030 - accuracy: 0.9139 - val_loss: 0.5310 - val_accuracy: 0.7703
Epoch 28/40
19/19 [==============================] - 7s 300ms/step - loss: 0.2704 - accuracy: 0.9172 - val_loss: 0.7975 - val_accuracy: 0.7297
Epoch 29/40
19/19 [==============================] - 7s 302ms/step - loss: 0.3850 - accuracy: 0.8581 - val_loss: 0.6020 - val_accuracy: 0.7973
Epoch 30/40
1

wandb: Adding directory to artifact (/content/wandb/run-20221012_075756-1fya4i00/files/model-best)... Done. 8.4s


19/19 [==============================] - 65s 4s/step - loss: 0.2982 - accuracy: 0.8902 - val_loss: 0.4275 - val_accuracy: 0.8446
Epoch 31/40
19/19 [==============================] - 8s 332ms/step - loss: 0.2923 - accuracy: 0.9071 - val_loss: 0.5526 - val_accuracy: 0.7770
Epoch 32/40
19/19 [==============================] - 7s 306ms/step - loss: 0.2640 - accuracy: 0.9071 - val_loss: 0.4391 - val_accuracy: 0.8378
Epoch 33/40
19/19 [==============================] - 7s 304ms/step - loss: 0.2688 - accuracy: 0.9223 - val_loss: 0.6048 - val_accuracy: 0.8243
Epoch 34/40
19/19 [==============================] - 7s 304ms/step - loss: 0.2139 - accuracy: 0.9341 - val_loss: 0.5489 - val_accuracy: 0.7838
Epoch 35/40
19/19 [==============================] - 7s 309ms/step - loss: 0.2982 - accuracy: 0.8953 - val_loss: 1.2091 - val_accuracy: 0.6149
Epoch 36/40
19/19 [==============================] - 7s 300ms/step - loss: 0.3907 - accuracy: 0.8463 - val_loss: 0.4558 - val_accuracy: 0.8243
Epoch 37/40
1

In [10]:
wandb.finish()

accuracy,▁▂▃▄▅▄▅▅▆▆▆▅▆▆▇▇▇▆▇▇▇▇▇▇▇███▇▇████▇▇█▇██
epoch,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
loss,█▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val_accuracy,▁▂▂▅▅▄▆▅▅▇▅▆▃▅▇▅▆▄▇▇▇▇█▇▇▆▆▆▇█▇█▇▇▃▇▆▇██
val_loss,█▂▃▂▂▂▂▁▂▁▂▁▂▂▁▁▁▂▁▁▁▁▁▁▁▁▁▂▁▁▁▁▁▁▃▁▁▁▁▁
accuracy,0.91047
best_epoch,29
best_val_loss,0.42748
epoch,39
loss,0.25224
val_accuracy,0.85135
